In [3]:
import numpy as np
import matplotlib 
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import random

In [67]:
# Function to generate the Dataset for a known hypothesis
# Assumptions: Range of values on the x and y axis are the same
#              Code is only for classifying data in 2D
def gen_data_perceptron(llim,ulim,target_weight_3D,N):
    
    """
    llim: a scalar defining the lower limit on the x and y axis
    ulim: a scalar defining the upper limit on the x and y axis
    target_weight: a column vector of 3x1 listing the known weights used for generating the target hypothesis
    N: Total number of data points
    Output: Returns a tuple with following elements at respective indices-
        index        element
        0 -          a numpy array of Nx3 with the first column being all 1s, second and third column 
                     consists of N points generated uniformly between llim and ulim
        1 -          column vector of Nx1, Target hypothesis corresponding to the target weight 
    """
    x1_sample = np.append(np.ones((N,1)),np.random.uniform(llim,ulim,N).reshape(N,1),axis=1)
    x2_sample = np.random.uniform(llim,ulim,N).reshape(N,1)
    data_partial = np.append(x1_sample,x2_sample,axis=1)    
    target = np.dot(data_partial,target_weight_3D)
    mask = target>0
    category_values = np.where(mask,np.ones((N,1)),-1)
    data_complete = np.append(data_partial,category_values,axis=1)
    
    return data_complete

# Function to compute the Linear Regression solution
def lin_reg(data_complete):
    """
    data_complete: A numpy array of Nx4 with the first column being all 1s, second and third column 
                consists of N points generated uniformly between llim and ulim and the last column
                contains the categories to be learnt
    
    Output: Returns a 3x1 column vector of final weights computed using the linear regression
    """
    a = np.linalg.inv(np.matmul(np.transpose(data_complete[:,0:3]),data_complete[:,0:3]))
    b = np.dot(np.transpose(data_complete[:,0:3]),data_complete[:,3].reshape(-1,1))
    final_weight = np.dot(a,b)
    return final_weight
    
# Function to extract the misclassified values 
def ident_misclass_pts(data_complete, weight_3D):
    """
    data_complete: A numpy array of Nx4 with the first column being all 1s, second and third column 
                consists of N points generated uniformly between llim and ulim and the last column
                contains the categories to be learnt
    weight_3D: A column vector of 3x1 specifying weights 
    
    Output: Returns a tuple with the following elements at the respective indices
    index   element
     index        element
        0 - a numpy array of Nx4 containing only the missclassified points and the 4 columns as above 
        1 - a scalar giving the proportion of missclassified points for the specified weight vector
    
    """
    ref_hyp_weight = np.dot(data_complete[:,0:3],weight_3D)
    mask = ref_hyp_weight > 0
    comp_vec = np.where(mask,np.ones((data_complete.shape[0],1)),-1)
    error_mask = comp_vec!=data_complete[:,3].reshape(-1,1)                    
    error_mask = np.append(error_mask,np.append(error_mask,np.append(error_mask,error_mask,axis=1),axis=1),axis=1)
    missclass_data_pts = data_complete[error_mask].reshape(-1,4)
    error_frac = missclass_data_pts.shape[0]/data_complete.shape[0]
    return (missclass_data_pts,error_frac)

# Function to update the weights for PLA using a misclassified point
def weight_update(weight_current_3D, missclass_pt):
    """
    weights_3D : a 3x1 column vector specifying the current set of weights in 3D
    missclass_pt: a missclassified point of shape (4,) with the last entry being the original category 
    
    output: returns a column vector of updated weights in 3D
    """
    if missclass_pt[3]==1:
        weight_updated = weight_current_3D + missclass_pt[0:3].reshape(3,1)
    else:
        weight_updated = weight_current_3D - missclass_pt[0:3].reshape(3,1)
    return weight_updated

# Perceptron Learning Algorithm
def PLA(data_complete, weight_3D_init):
    """
    data_complete : 
    weights_3D_init : a 3x1 column vector specifying the initial set of weights in 3D 
    data_complete: A numpy array of Nx4 with the first column being all 1s, second and third column 
                   consists of N points generated uniformly between llim and ulim and the last column
                   contains the categories to be learnt
                   
    output: Returns a tuple with following elements at respective indices-
        index        element
        0 -          a column vector 3x1 containing the final weights
        1 -          number of iterations until convergence
    """
    missclass_dataset = ident_misclass_pts(data_complete, weight_3D_init)[0]
    no_of_iterations=0
    while (missclass_dataset.size!=0):
        n = random.randint(0,missclass_dataset.shape[0]-1)
        weight_3D_init = weight_update(weight_3D_init, missclass_dataset[n])
        missclass_dataset = ident_misclass_pts(data_complete, weight_3D_init)[0]
        no_of_iterations = no_of_iterations + 1
    final_weight = weight_3D_init
    #visual_perceptron(data_complete, final_weight)
    return (final_weight,no_of_iterations)

In [68]:
# Computing the average Ein and Eout
nsims = 1000
Ein = 0
Eout = 0
llim = -1
ulim = 1
no_of_iterations = 0
N_reg = 100
N_pla = 10

In [29]:
# Computing the average in-sample and out of sample error
for i in range(nsims):
    target_weight_3D = np.array([[np.random.uniform(-1,1)],[np.random.uniform(-1,1)],[np.random.uniform(-1,1)]])
    data_complete = gen_data_perceptron(llim,ulim,target_weight_3D,N_reg)
    data_complete_test = gen_data_perceptron(llim,ulim,target_weight_3D,10*N_reg)
    
    # Computing the weights using Linear Regression Algorithm and the Ein for this step
    final_weight_reg = lin_reg(data_complete)
    Ein = Ein + ident_misclass_pts(data_complete, final_weight_reg)[1]
    Eout = Eout + ident_misclass_pts(data_complete_test, final_weight_reg)[1]
    
print("The average in sample error is:", Ein/nsims)
print("The average out of sample error is:", Eout/nsims)

The average in sample error is: 0.033379999999999986
The average out of sample error is: 0.039771


In [69]:
# Using Regression as the starting point for PLA
for i in range(nsims):
    target_weight_3D = np.array([[np.random.uniform(-1,1)],[np.random.uniform(-1,1)],[np.random.uniform(-1,1)]])
    data_complete = gen_data_perceptron(llim,ulim,target_weight_3D,N_pla)
    
    # Computing the weights using Linear Regression Algorithm and using them as initial weights for PLA
    final_weight_reg = lin_reg(data_complete)
    pla_results = PLA(data_complete, final_weight_reg)
    no_of_iterations = no_of_iterations + pla_results[1]
print("The average number of iterations for PLA to converge:", no_of_iterations/nsims)

The average number of iterations for PLA to converge: 3.161
